# Mentor dashboard analytics
This notebook supports the [Superset dashboard](https://superset.wikimedia.org/superset/dashboard/294/) used to analyze the mentor dashboard.

### License statement
Copyright 2021 Martin Urbanec (murbanec-ctr@wikimedia.org)

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at

http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

In [ ]:
from wmfdata import spark, hive, mariadb, utils
import requests
import pandas as pd

In [ ]:
sparkSession = spark.get_session(app_name='Mentor dashboard analysis')

In [ ]:
PILOT_WIKIS = list(
    set(utils.get_dblist('growthexperiments')) -
    {'testwiki'}
)

## Update `urbanecm_growth.mentor_count`
The `urbanecm_growth.mentor_count` Hive table is used by the Superset dashboard

In [ ]:
def get_mentor_count(domain, page_title):
    r = requests.get('https://{domain}/w/api.php'.format(domain=domain), params={
        "action": "query",
        "format": "json",
        "prop": "links",
        "titles": page_title,
        "plnamespace": "2",
        "pllimit": "max"
    })
    data = r.json()
    data = data['query']['pages']
    data = data[list(data.keys())[0]]
    try:
        return len(data['links'])
    except:
        return 0

In [ ]:
wikis_df = spark.run('SELECT * FROM canonical_data.wikis')
growth_wikis = utils.get_dblist('growthexperiments')
pandaData = {
    'domain': [],
    'auto_mentors': [],
    'manual_mentors': [],
    'mentors': []
}
for wiki in growth_wikis:
    domain = None
    tmp = list(wikis_df.loc[wikis_df.database_code == wiki].domain_name)
    if len(tmp) == 0:
        continue
    domain = tmp[0]
    
    try:
        geConfig = requests.get(
            'https://{domain}/w/index.php?title=MediaWiki:GrowthExperimentsConfig.json&action=raw'.format(
                domain=domain
            )
        ).json()
    except:
        print('ERROR fetching wiki config: {wiki}'.format(wiki=wiki))
        continue
    
    autoMentorList = geConfig.get('GEHomepageMentorsList', '')
    manualMentorList = geConfig.get('GEHomepageManualAssignmentMentorsList', '')
    
    autoMentorsCount = 0
    if autoMentorList != '' and autoMentorList is not None:
        autoMentorsCount = get_mentor_count(domain, autoMentorList)
    
    manualMentorsCount = 0
    if manualMentorList != '' and manualMentorList is not None:
        manualMentorsCount = get_mentor_count(domain, manualMentorList)
    
    pandaData['domain'].append(domain)
    pandaData['auto_mentors'].append(autoMentorsCount)
    pandaData['manual_mentors'].append(manualMentorsCount)
    pandaData['mentors'].append(autoMentorsCount + manualMentorsCount)
    
mentors_df = pd.DataFrame(pandaData)

In [ ]:
mentors_df.to_csv('/home/urbanecm/tmp/mentors.tsv', index=False)

In [ ]:
!sed 1d -i /home/urbanecm/tmp/mentors.tsv

In [ ]:
hive.run([
    'TRUNCATE TABLE urbanecm_growth.mentor_count',
    '''
    LOAD DATA LOCAL INPATH "/home/urbanecm/tmp/mentors.tsv"
    OVERWRITE INTO TABLE urbanecm_growth.mentor_count
    '''
])

In [ ]:
hive.load_csv(
    path='/home/urbanecm/tmp/mentors.tsv',
    field_spec='domain string, auto_mentors int, manual_mentors int, mentors int',
    db_name='urbanecm_growth',
    table_name='mentor_dashboard_mentor_count'
)

## Update `urbanecm_growth.mentor_dashboard_deployed_wikis`
The `urbanecm_growth.mentor_dashboard_deployed_wikis` Hive table is used by Superset dashboard to show data only for the pilot wikis.

In [ ]:
spark.run([
    'TRUNCATE TABLE urbanecm_growth.mentor_dashboard_deployed_wikis',
    '''
    INSERT INTO urbanecm_growth.mentor_dashboard_deployed_wikis
    SELECT domain_name
    FROM canonical_data.wikis
    WHERE
        -- we need to be deployed on the wiki in the first place
            database_code IN ({pilot_wikis})
        -- wiki needs to have at least one mentor to be meaningful
        AND (
            domain_name IN (
                SELECT domain
                FROM urbanecm_growth.mentor_count
                WHERE mentors > 0
            )
        )
        -- 
    '''.format(
        pilot_wikis=', '.join(["'%s'" % x for x in PILOT_WIKIS])
    ),
    'SELECT * FROM urbanecm_growth.mentor_dashboard_deployed_wikis'
])

## Update `urbanecm_growth.mentor_dashboard_pageviews_views_per_day`

In [ ]:
spark.run([
    '''
    INSERT OVERWRITE TABLE urbanecm_growth.mentor_dashboard_pageviews_views_per_day
    SELECT
        tmp.date,
        tmp.domain,
        user_id
    FROM (
        SELECT `date`, domain
        FROM urbanecm_growth.mentor_dashboard_deployed_wikis
        CROSS JOIN (
            SELECT DISTINCT DATE(dt) AS `date`
            FROM event.mediawiki_mentor_dashboard_visit
            WHERE (
                year = 2021 AND
                month >= 10
            ) OR (
                year > 2021
            )
        ) AS dates
    ) AS tmp
    LEFT JOIN event.mediawiki_mentor_dashboard_visit AS views ON (
        (tmp.date=DATE(views.dt)) AND
        (tmp.domain=meta.domain)
    )
    ''',
    '''
    SELECT *
    FROM urbanecm_growth.mentor_dashboard_pageviews_views_per_day
    LIMIT 10'''
])